# Import libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
import xml.etree.ElementTree as ET
import re
import random
import time
import os
import pandas as pd

# Get URL from  Sitemap

In [ ]:
sitemap_url = 'https://myanimelist.net/sitemap/anime-001.xml'

# Tải về file XML
response = requests.get(sitemap_url)

# Nếu thành công, phân tích cú pháp file XML
if response.status_code == 200:
    sitemap_xml = response.content
    
    # Phân tích cú pháp XML
    root = ET.fromstring(sitemap_xml)
    
    # Tìm tất cả các URL trong sitemap
    urls = []
    for sitemap in root.findall('{http://www.sitemaps.org/schemas/sitemap/0.9}url'):
        loc = sitemap.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc').text
        urls.append(loc)
else:
    print('Cannot get sitemap, Status code: {response.status_code}')

In [ ]:
# Write urls to file
with open('anime_urls.txt','w', encoding='utf-8') as f:
    for url in urls:
        f.write(url + '\n')

In [ ]:
len(urls)

# Crawl data from URLS

In [34]:
# Khởi tạo WebDriver cho trình duyệt Chrome
driver_path = r"C:\Users\Admin\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)

C:\Users\Admin\AppData\Local\Temp\ipykernel_17924\768845302.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


In [3]:
# Read urls from file
with open('anime_urls.txt','r',encoding='utf-8') as f:
    urls = [line.strip() for line in f]

In [4]:
# Hàm lấy thông tin 1 anime từ url
def crawl_anime_details(anime_url):
    driver.get(anime_url)
    time.sleep(random.uniform(1,3))
    
    # Lấy thông tin của anime
    # 1. ID
    try:
        anime_link = driver.find_element(By.XPATH, '//a[contains(@href, "animeid=")]')
        href_value = anime_link.get_attribute('href')
        anime_id = re.search(r'animeid=(\d+)', href_value).group(1)
    except Exception as e:
        anime_id = None
        
    # 2. Title
    try:
        title = driver.find_element(By.CLASS_NAME, 'title-name').text
    except Exception as e:
        title = None
    
    # 3. Eposides
    try:
            # Tìm phần tử cha chứa số episodes
        episodes_element = driver.find_element(By.XPATH, '//span[text()="Episodes:"]/parent::div')
            # Lấy toàn bộ văn bản trong phần tử
        episodes_text = episodes_element.text
            # Tách riêng số episodes từ văn bản (bỏ đi "Episodes:")
        episodes = episodes_text.replace("Episodes:", "").strip()
    except Exception as e:
        episodes = None
    
    # 4. Type
    try:
        types_element = driver.find_element(By.XPATH, '//span[text()="Type:"]/parent::div')
        types_text = types_element.text
        types = types_text.replace("Type:", "").strip()
    except Exception as e:
        types = None
    
    # 5. Aired
    try:
        aired_element = driver.find_element(By.XPATH, '//span[text()="Aired:"]/parent::div')
        aired_text = aired_element.text
        aired = aired_text.replace("Aired:", "").strip()
    except Exception as e:
        aired = None
    
    # 6. Producers
    try:
        producers_element = driver.find_element(By.XPATH, '//span[text()="Producers:"]/parent::div')
        producers_text = producers_element.text
        producers = producers_text.replace("Producers:", "").strip()
    except Exception as e:
        producers = None
    
    # 7. Studios
    try:
        studios_element = driver.find_element(By.XPATH, '//span[text()="Studios:"]/parent::div')
        studios_text = studios_element.text
        studios = studios_text.replace("Studios:", "").strip()
    except Exception as e:
        studios = None
    
    # 8. Source
    try:
        source_element = driver.find_element(By.XPATH, '//span[text()="Source:"]/parent::div')
        source_text = source_element.text
        source = source_text.replace("Source:", "").strip()
    except Exception as e:
        source = None
    
    # 9. Genres
    try: 
        genres_element = driver.find_element(By.XPATH, '//span[text()="Genres:"]/parent::div')
        genres_text = genres_element.text
        genres = genres_text.replace("Genres:", "").strip()
    except Exception as e:
        genres = None
    
    # 10. Duration
    try: 
        duration_element = driver.find_element(By.XPATH, '//span[text()="Duration:"]/parent::div')
        duration_text = duration_element.text
        duration = duration_text.replace("Duration:", "").strip()
    except Exception as e:
        duration = None
    
    # 11. Score
    try:
        score = driver.find_element(By.CLASS_NAME, 'score-label').text
    except Exception as e:
        score = None
    
    # 12. Rating count
    try:
                # Tìm phần tử chứa ratingCount
        rating_count_element = driver.find_element(By.XPATH, '//span[@itemprop="ratingCount"]')
            # Dùng JavaScript để lấy textContent của phần tử bị ẩn
        rating_count = driver.execute_script("return arguments[0].textContent;", rating_count_element)
    except Exception as e:
        rating_count = None
        
    # 13. Rank
    try:
        rank_element = driver.find_element(By.XPATH, '//span[text()="Ranked:"]/parent::div')
        rank_text = rank_element.text
        rank = rank_text.replace("Ranked: #", "").strip()
    except Exception as e:
        rank = None
        
    # 14. Popularity
    try:
        popularity_element = driver.find_element(By.XPATH, '//span[text()="Popularity:"]/parent::div')
        popularity_text = popularity_element.text
        popularity = popularity_text.replace("Popularity: #", "").strip()
    except Exception as e:
        popularity = None

    # Trả về thông tin đã cào
    return{
        "ID": anime_id,
        "Title": title,
        "Episodes": episodes,
        "Type": types,
        "Aired": aired,
        "Producers": producers,
        "Studios": studios,
        "Source": source,
        "Genres": genres,
        "Duration": duration,
        "Score": score,
        "Rating count": rating_count,
        "Rank": rank,
        "Popularity": popularity 
    }

In [32]:
animes = []

In [6]:
n = len(urls)
n

25618

In [42]:
# 
i-=1

In [43]:

urls[i]

'https://myanimelist.net/anime/57179/No5'

In [35]:
check_error_page = 0
while i<n:
    while i<n:
        anime = crawl_anime_details(urls[i])
        print(anime['ID'])
        if anime['ID'] == None:
            break
        i += 1
        animes.append(anime)
    check_error_page += 1
    if check_error_page == 2:
        i+= 1
        check_error_page = 0

56093
56097
56105
56106
56107
56108
56109
56110
56111
56122
56123
56129
56132
56133
56135
56142
56144
56145
56146
56148
None
None
56153
56154
56156
56157
56158
56159
56163
56164
56165
56168
56169
56170
56172
56175
56176
56178
56179
56182
56183
56184
56186
56189
56190
56191
56192
56195
56196
56199
56200
56201
56206
56210
56213
56218
56228
56230
56234
56235
56236
56238
56239
56240
56242
56243
56246
56247
56259
56261
56262
56263
56264
56265
56266
56267
56268
56269
56270
56271
56272
56273
56274
56275
56276
56277
56278
56279
56280
56281
56282
56283
56284
56287
56290
56291
56292
56293
56294
56295
56296
56297
56298
56299
56300
56301
56302
56303
56304
56305
56306
56307
56308
56309
56310
56311
56312
56315
56317
56318
56319
56320
56321
56322
56323
56332
56333
56334
56335
56336
56337
56338
56339
56342
56345
56346
56347
56348
56352
56354
56356
56357
56358
56359
56360
56361
56362
56363
56364
56365
56366
56368
56369
56370
56371
56372
56373
56374
56375
56376
56378
56379
56380
56381
56383
56384
56385


In [36]:
len(animes)

669

# Export to CSV file

In [37]:


file_path = 'animes.csv'

#df = pd.DataFrame(animes)
df_new = pd.DataFrame(animes)

if not os.path.isfile(file_path):
    df.to_csv(file_path, index = False, mode = 'w', encoding = 'utf-8')
else:
    df_new.to_csv(file_path, index = False, mode = 'a', header = False, encoding = 'utf-8')
    

In [ ]:
file_path = 'animes.csv'

df = pd.DataFrame(animes)
df.to_csv(file_path, index = False, mode = 'w', encoding = 'utf-8')

In [ ]:
animes